In [1]:
import numpy as np

In [117]:
import matlab
import matlab.engine
eng = matlab.engine.start_matlab()
eng.addpath(r'C:/Users/igorp/OneDrive/Documents/GitHub/Project-Heart/matlab',nargout=0)

In [3]:
import os
from pathlib import Path
import numpy as np
import pyvista as pv
# import project_heart as ph
from project_heart.modules.geometry import Geometry
pv.set_jupyter_backend("pythreejs")
from project_heart.lv import LV
from project_heart.enums import *

In [5]:
# dir = Path("C:/Users/igorp/University of South Florida/Wenbin Mao - Igor/LV_Meshes/Heart_models")
# # dir = Path("C:/Users/igornobrega/USF_Research/University of South Florida/Wenbin Mao - Igor/LV_Meshes/Heart_models")
# files = os.listdir(dir)

# file_id = 3 # 5 has a problem

print("File:", "C:/Users/igorp/OneDrive/Desktop/sample_lv_1.xplt")

lv = LV()
lv.from_xplt("C:/Users/igorp/OneDrive/Desktop/sample_lv_1.xplt", identifier="elemTag", threshold=[0, 1])

plotter = pv.Plotter(lighting='three lights')
plotter.background_color = 'w'
plotter.enable_anti_aliasing()
plotter.add_mesh(lv.mesh, 
                  cmap="Set2", 
                  opacity=1.0, 
                  show_edges=False,
                  ambient=0.2, 
                  diffuse=0.5, 
                  specular=0.5, 
                  specular_power=90,
                  #  smooth_shading=True,
                  )
plotter.enable_shadows()
plotter.show()

File: C:/Users/igorp/OneDrive/Desktop/sample_lv_1.xplt


Renderer(camera=PerspectiveCamera(aspect=1.3333333333333333, children=(DirectionalLight(color='#fefefe', posit…

In [6]:
lv.identify_base_and_apex_regions()

(array([0., 0., 0., ..., 0., 0., 0.]), array([0., 0., 0., ..., 0., 0., 0.]))

In [137]:
import tetgen
tet = tetgen.TetGen(lv.get_surface_mesh().triangulate())
tet.tetrahedralize(order=1, mindihedral=20, minratio=1.5)
lv_tet = LV()
lv_tet.from_pyvista_dataset(tet.grid)
lv_tet

In [138]:
tet.grid

UnstructuredGrid,Information
N Cells,294073
N Points,59582
X Bounds,"-3.948e+01, 3.455e+01"
Y Bounds,"-3.831e+01, 3.370e+01"
Z Bounds,"5.064e-02, 9.641e+01"
N Arrays,0


In [140]:
lv_tet.mesh

UnstructuredGrid,Information
N Cells,294073
N Points,59582
X Bounds,"-3.948e+01, 3.455e+01"
Y Bounds,"-3.831e+01, 3.370e+01"
Z Bounds,"5.064e-02, 9.641e+01"
N Arrays,0


In [139]:
lv_tet.get_surface_mesh()

PolyData (0x24003d51460)
  N Cells:	45098
  N Points:	22551
  X Bounds:	-3.948e+01, 3.455e+01
  Y Bounds:	-3.831e+01, 3.370e+01
  Z Bounds:	5.064e-02, 9.641e+01
  N Arrays:	2

In [126]:
lv_tet.mesh

UnstructuredGrid (0x24007d15fa0)
  N Cells:	68763
  N Points:	21906
  X Bounds:	-3.948e+01, 3.455e+01
  Y Bounds:	-3.831e+01, 3.370e+01
  Z Bounds:	5.064e-02, 9.641e+01
  N Arrays:	2

In [141]:
plotter = pv.Plotter(lighting='three lights')
plotter.background_color = 'w'
plotter.enable_anti_aliasing()
plotter.add_mesh(lv_tet.mesh, 
                  cmap="Set2", 
                  opacity=1.0, 
                  show_edges=True,
                  ambient=0.2, 
                  diffuse=0.5, 
                  specular=0.5, 
                  specular_power=90,
                  #  smooth_shading=True,
                  )
plotter.enable_shadows()
plotter.show()

Renderer(camera=PerspectiveCamera(aspect=1.3333333333333333, children=(DirectionalLight(color='#fefefe', posit…

In [87]:
# from project_heart.utils.vector_utils import *
# lvsurf = lv_tet.get_surface_mesh()
# pts = lvsurf.points
# # get surface Normals and get respective points-data
# lvsurf.compute_normals(inplace=True)
# surf_normals = lvsurf.get_array("Normals", "points")
# # est geometry center
# center = np.mean(pts, axis=0)
# # get vector from pts at surface to center
# pts_to_center = center - pts
# # compute angle difference between surface normals and pts to center
# angles = angle_between(pts_to_center, surf_normals,
#        check_orientation=False)  # returns [0 to pi]
# # set initial endo-epi surface guess
# thresh = np.radians(90)
# endo_ids = np.where(angles < thresh)[0]
# epi_ids = np.where(angles >= thresh)[0]
# surf_to_global = lvsurf.point_data["vtkOriginalPointIds"]
# tet_est_surfs = np.zeros(lv_tet.mesh.n_points)
# tet_est_surfs[surf_to_global[endo_ids]] = 1
# tet_est_surfs[surf_to_global[epi_ids]] = 2
# lv_tet.mesh.point_data["EPI_ENDO"] = tet_est_surfs
# lv_tet.mesh = lv_tet.mesh.point_data_to_cell_data()

In [147]:
lv_tet.to_vtk("tet.vtk")

In [82]:
domains = np.array(lv_tet.mesh.cell_data["EPI_ENDO"])
domains[domains<=1.25] = 1.0
domains[domains>=1.75] = 2.0
# domains += 1
domains = domains.astype(np.int64)
domains

array([1, 1, 1, ..., 1, 1, 1], dtype=int64)

In [83]:
np.unique(domains)

array([1, 2], dtype=int64)

In [84]:
tet_cells.T

array([[19974, 20639, 17205, ...,  5445, 10643,  9663],
       [ 9985, 20577, 17208, ...,  5444, 10495, 20412],
       [19905, 20640,  3201, ..., 15159, 19951, 20504],
       [19906,  9530, 17206, ...,  5446, 19949,  9662]], dtype=int64)

In [143]:
tet_nodes = lv_tet.points()
tet_cells = lv_tet.cells(as_json_ready=True)["TETRA"]+1
# tet_doms = np.ones(len(tet_cells), dtype=np.int64)

In [144]:
np.savetxt("nodes.csv", tet_nodes, delimiter=",")
np.savetxt("elems.csv", tet_cells.astype(np.int32), fmt='%i', delimiter=",")

In [145]:
tet_nodes = lv_tet.points()
tet_cells = lv_tet.cells(as_json_ready=True)["TETRA"]+1
# tet_doms = np.ones(len(tet_cells), dtype=np.int64)

# tet_doms[:2] = 2

tet_mesh = {
  "NODES": matlab.double(tet_nodes), 
  "ELEMENTS": matlab.int64(tet_cells),
  # "DOMAINS": matlab.int64(tet_doms)
  }

In [146]:
eng.calc_fiber_orientation(
  tet_mesh,
  tet_mesh,
  matlab.double(lv.get_virtual_node(LV_VIRTUAL_NODES.APEX.value)),
  matlab.double(lv.get_virtual_node(LV_VIRTUAL_NODES.BASE.value)),
  -60.0,
  60.0
)

MatlabExecutionError: Output argument "f0" (and possibly others) not assigned a value in the execution with "calc_fiber_orientation" function.


In [ ]:
cells = lv.cells(as_json_ready=True)["TETRA"]+1
cells

In [ ]:
np.arange(1, 1+len(cells)).reshape((-1,1))

In [ ]:
np.hstack((np.arange(1, 1+len(cells)).reshape((-1,1)), cells)).shape